## Exercise 4

For this exercise you are given a database containing information about real estate around
Uppsala. You are also given a Python module that helps working the database.

- Download the [database](...) and the [code `db.py`](...)
- Read the module's [documentation](../../files/db_module_documentation.html)

The module `db.py` will help you produce a HTML file whit your results plotted on a map. This set-up uses Google Maps, so you will need to fetch an [API key](https://developers.google.com/maps/documentation/javascript/get-api-key). If you do not have a Google account, or don’t want to bother with the API key, you can always content yourself with the print function.

![result](../../img/map.jpg)

### Questions:


a) What is the price of the cheapest house around a given center and within 2 kilometers?

The center is defined as the point:

latitude: 59.865795990339876

longitude: 17.64583576202392


b) Find the most expensive hous per square meter